# Tạo **Span Adaptation Vectors**

- **Mục đích**
  - Từ **Adaptation Vector theo token** (đã tính trước) + **mô hình fine-tuned**, tạo **vector span** cho **từng argument (ARG-k)** trong câu.
  - Lưu **mỗi argument** thành **một file riêng** để tiện xây DB và truy vấn sau này.

- **Đầu vào**
  - Mô hình fine-tuned (để suy luận nhãn token + lấy **classifier weights**):
    ```
    .../Finetuned_Models/biobert-srl-best-model
    ```
  - Dữ liệu gốc (JSON) chứa câu và `arguments`:
    ```
    .../Clean_Dataset/Corpus/Split_GramVar/{Train,Test}/*.json
    .../Clean_Dataset/Corpus/Split_ParaVE/{Train,Test}/*.json
    ```
  - **Adaptation vectors theo token** (đã chuẩn hoá & align trước đó):
    ```
    .../adaptation_vectors_{train|test}_{gramvar|parave}_pt_aligned/<verb_name>/sentence_i.pt
    ```
    > Mỗi `.pt` là dict: `{ "layer_0": Tensor[num_tokens, 768], ..., "layer_12": ... }`

- **Đầu ra**
  - Thư mục **Span AV per-argument**:
    ```
    span_adaptation_vectors_train_gramvar/
    span_adaptation_vectors_train_parave/
    span_adaptation_vectors_test_gramvar/
    span_adaptation_vectors_test_parave/
    ```
  - Mỗi câu/động từ sinh **nhiều file** (mỗi ARG một file):
    ```
    <output_root>/<verb_name>/sentence_<i>_ARG_0.pt
    <output_root>/<verb_name>/sentence_<i>_ARG_1.pt
    ...
    ```
  - Nội dung **mỗi file**: dict theo lớp  
    `{ "layer_0": Tensor[2304], "layer_1": Tensor[2304], ... }`  
    với 2304 = **768(begin) ⊕ 768(end) ⊕ 768(content)**

- **Cách tính Span AV cho một argument**
  1. **Tokenize** toàn câu → `full_tokens`
  2. **Dự đoán nhãn token** bằng mô hình fine-tuned:
     - `predictions = argmax(logits)`
     - Lấy **classifier weights**: `W ∈ R[num_labels, 768]`
  3. **Định vị argument** (`arg_text`) trong `full_tokens`  
     → `(start_idx, end_idx)` (dịch +1 vì có **[CLS]**)
  4. Với **mỗi lớp `k`**:
     - Lấy **token-AV** dải span: `X_k[start:end]`
     - Với mỗi token `t` trong span, tính **mức đóng góp**:
$$
a_t = \max\Big(0,\; \frac{\langle \text{AV}_t,\; w_{r^*}\rangle}{\|w_{r^*}\| + \varepsilon}\Big),
\quad r^* = \text{prediction}(t)
$$
     - Chuẩn hoá trọng số: $ w_t = \frac{a_t}{\sum_j a_j + \varepsilon} $
     - **Content vector**: $ \text{content} = \sum_t w_t \cdot \text{AV}_t \in \mathbb{R}^{768} $
     - **Span vector lớp k**:
$$
\text{span}_k = \text{AV}_{\text{begin}} \;\oplus\; \text{AV}_{\text{end}} \;\oplus\; \text{content}
\;\in\; \mathbb{R}^{2304}
$$
  5. Ghép cho **mọi lớp** → lưu ra file `.pt` của **argument** đó.

- **Quy trình chạy**
  1. Tải **tokenizer** + **mô hình fine-tuned** (eval, dùng GPU nếu có).
  2. Với từng **tập** (Train/Test × GramVar/ParaVE):
     - Duyệt tất cả file JSON (mỗi file = 1 động từ).
     - Với mỗi câu:
       - Nạp `sentence_i.pt` (token-AV).
       - Tạo **Span AV per-argument**.
       - **Lưu từng ARG**: `sentence_i_ARG_k.pt` (đổi `-` → `_` cho tên file an toàn).

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification

# THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH

# Tự động chọn GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Tải lại mô hình fine-tuned để lấy trọng số classifier và chạy dự đoán
print("Đang tải lại mô hình Fine-tuned...")
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')
tokenizer = AutoTokenizer.from_pretrained(final_model_path)
model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)
model_ft.eval()
print(" -> Tải mô hình thành công!")

# --- DANH SÁCH CÁC BỘ DỮ LIỆU CẦN XỬ LÝ ---
datasets_to_process = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_gramvar_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_gramvar')
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_parave_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_parave')
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Test/adaptation_vectors_test_gramvar_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content/span_adaptation_vectors_test_gramvar')
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_test_parave_pt_aligned'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content/span_adaptation_vectors_test_parave')
    ),
]

# CÁC HÀM XỬ LÝ

def find_sub_list(sl, l):
    """Tìm vị trí của một list con (sl) trong một list lớn (l)."""
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e == sl[0]):
        if l[ind:ind+sll] == sl:
            return ind, ind + sll - 1
    return -1, -1

def create_span_vectors(sentence_text, arguments, token_av_data, model, tokenizer):
    """
    Tạo span adaptation vector cho tất cả các argument trong một câu.
    """
    full_tokens = tokenizer.tokenize(sentence_text)

    with torch.no_grad():
        inputs = tokenizer(sentence_text, return_tensors="pt").to(device)
        logits = model(**inputs).logits
        predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()

    classifier_weights = model.classifier.weight.detach().cpu()

    span_vectors_all_args = {}

    for arg_label, arg_text in arguments.items():
        if not arg_label.startswith('ARG') or not arg_text:
            continue

        arg_tokens = tokenizer.tokenize(arg_text)
        start_idx, end_idx = find_sub_list(arg_tokens, full_tokens)

        start_idx += 1
        end_idx += 1

        if start_idx == 0:
            continue

        span_vectors_per_layer = {}
        for layer_name, layer_av_tensor in token_av_data.items():
            layer_av_tensor = layer_av_tensor.cpu()
            adap_vec_begin = layer_av_tensor[start_idx]
            adap_vec_end = layer_av_tensor[end_idx]
            span_token_avs = layer_av_tensor[start_idx : end_idx + 1]
            span_token_preds = predictions[start_idx : end_idx + 1]

            a_k_list = []
            for i in range(len(span_token_avs)):
                adap_vec_tk = span_token_avs[i]
                pred_label_id = span_token_preds[i]
                w_r_star = classifier_weights[pred_label_id]

                dot_product = torch.dot(adap_vec_tk, w_r_star)
                norm_w = torch.linalg.norm(w_r_star)
                a_k = torch.max(torch.tensor(0.0), dot_product / (norm_w + 1e-8))
                a_k_list.append(a_k)

            sum_a_k = torch.sum(torch.stack(a_k_list)) + 1e-8

            content_vec = torch.zeros(768, dtype=torch.float32)
            if sum_a_k > 1e-8:
                for i in range(len(span_token_avs)):
                    w_k = a_k_list[i] / sum_a_k
                    content_vec += w_k * span_token_avs[i]

            span_vec = torch.cat([adap_vec_begin, adap_vec_end, content_vec])
            span_vectors_per_layer[layer_name] = span_vec

        span_vectors_all_args[arg_label] = span_vectors_per_layer

    return span_vectors_all_args

# THỰC THI CHƯƠNG TRÌNH

if __name__ == "__main__":
    print("\n--- Bắt đầu quá trình tạo Span Adaptation Vector ---")

    for name, data_dir, av_input_dir, output_dir in datasets_to_process:
        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f"=================================================")

        os.makedirs(output_dir, exist_ok=True)
        json_files = glob.glob(os.path.join(data_dir, '*.json'))

        if not json_files:
            print(f"CẢNH BÁO: Không tìm thấy file dữ liệu gốc nào trong '{data_dir}'. Bỏ qua.")
            continue

        pbar = tqdm(total=len(json_files), desc=f"Xử lý {name}", unit="file")

        for json_file in json_files:
            verb_name = os.path.basename(json_file).replace('_test_set.json', '').replace('_train_set.json', '')

            with open(json_file, 'r', encoding='utf-8') as f:
                original_data = json.load(f)

            output_verb_dir = os.path.join(output_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            for i, item in enumerate(original_data):
                sentence_text = item.get('text')
                arguments = item.get('arguments')

                if not sentence_text or not arguments:
                    continue

                try:
                    av_path = os.path.join(av_input_dir, verb_name, f"sentence_{i}.pt")
                    token_av_data = torch.load(av_path)

                    # span_av_data_per_arg là một dict, ví dụ: {"ARG-0": {"layer_0":...}, "ARG-1": ...}
                    span_av_data_per_arg = create_span_vectors(sentence_text, arguments, token_av_data, model_ft, tokenizer)

                    # Lặp qua từng argument và lưu thành file riêng
                    if span_av_data_per_arg:
                        for arg_label, vectors_for_this_arg in span_av_data_per_arg.items():
                            # Tạo tên file mới, ví dụ: sentence_0_ARG-0.pt
                            safe_arg_label = arg_label.replace('-', '_') # Thay thế ký tự không an toàn
                            output_filename = f"sentence_{i}_{safe_arg_label}.pt"
                            output_path = os.path.join(output_verb_dir, output_filename)

                            # Lưu dữ liệu chỉ của argument này
                            # vectors_for_this_arg là dict {"layer_0": ..., "layer_1": ...}
                            torch.save(vectors_for_this_arg, output_path)

                except FileNotFoundError:
                    pass
                except Exception as e:
                    pbar.write(f"Lỗi khi xử lý câu {i}, động từ {verb_name}: {e}")

            pbar.update(1)
        pbar.close()

    print("\n--- QUÁ TRÌNH TẠO SPAN ADAPTATION VECTOR ĐÃ HOÀN TẤT! ---")



Sử dụng thiết bị: cuda
Đang tải lại mô hình Fine-tuned...
 -> Tải mô hình thành công!

--- Bắt đầu quá trình tạo Span Adaptation Vector ---

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_GramVar


Xử lý Test_GramVar:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_ParaVE


Xử lý Test_ParaVE:   0%|          | 0/35 [00:00<?, ?file/s]


--- QUÁ TRÌNH TẠO SPAN ADAPTATION VECTOR ĐÃ HOÀN TẤT! ---


- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../adaptation_vectors_train_gramvar_pt_aligned/
  └── bind/
      ├── sentence_0.pt
      └── sentence_1.pt

  .../span_adaptation_vectors_train_gramvar/
  └── bind/
      ├── sentence_0_ARG_0.pt
      ├── sentence_0_ARG_1.pt
      └── sentence_1_ARG_0.pt